In [10]:
import sys
import os
import re
import datetime
import json

sys.path.append(os.getcwd())

from backend import config
from backend.modelClasses import PromptHandler, LLMWrapper, TeacherAgent
from backend.tts import TTSProcessor
from backend.pipeline import load_scenario, run_pipeline
from backend.utils import reformat_transcript_to_list
from backend.prompts.scenarios import *


### Scenario Prep

In [11]:
# Read the scenario from a file
scenario_file_path = "./backend/prompts/scenarios/cafe.txt"
with open(scenario_file_path, "r", encoding="utf-8") as file:
    scenario = file.read()
 

### Transcript Generation

In [12]:
# Create the prompt handler and transcript generator instances
prompt_handler = PromptHandler(config.SYSTEM_PROMPT_TEMPLATE_PATH)
transcript_generator = LLMWrapper(config.MODEL_NAME, temperature=0.0)


language = "French"
country_name = "France"
characters = config.CHARACTER_CONFIG[language][country_name]
character_1 = characters["speaker_1"]
character_2 = characters["speaker_2"]


# Format the prompt and generate the transcript
prompt = prompt_handler.format_prompt(
    scenario=scenario,
    country_name=country_name,
    language=language,
    character_1_name=character_1["name"],
    character_2_name=character_2["name"]
)
print("----- Prompt -----")
print(prompt)
raw_transcript = transcript_generator.generate(prompt)

print("----- Raw Transcript -----")
print(raw_transcript)


----- Prompt -----
You are an expert content creator for French language learners. Your role is to produce a **natural-sounding, French-only conversation** between two characters, strictly following the formatting rules.

## **Scenario:**  
The conversation must be based on the following scenario: **A tourist enters a busy cafe and goes up to the counter to order. 
They don't know exactly what to order and want some advice from the server. 
After ordering and paying with card they take a seat and wait for their order to arrive. 
Once it arrives they interact with the server and finsih their order and then leave.**.

## **Requirements:**
1. **French Only**
   - **Do not include any English text** except for character names.
   - Reflect the **local dialect** and **culture** of France in your French.

2. **Fixed Characters**
   - You MUST use ONLY these character names for France:
     - Charles
     - Sophie
   - Do NOT introduce any other characters
   - Do NOT change or modify these n

In [13]:
# Create the teacher agent instance
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_PROMPT_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.0,
)

# Generate the explained transcript using keyword arguments for additional context
explained_transcript = teacher_agent.explain(
    raw_transcript, 
    scenario=scenario, 
    country_name=country_name,
    language=language,
    character_1_name=character_1["name"],  
    character_2_name=character_2["name"]   
)

print("----- Explained Transcript -----")
print(explained_transcript)


----- Explained Transcript -----
Charles: "Bonjour, madame. Je suis un touriste et je suis un peu perdu dans votre menu. Pouvez-vous me donner quelques conseils?"

Sophie: "Bien sûr, monsieur. Nous avons des croissants fraîchement cuits, des pains au chocolat et des escargots en casque. Quelque chose vous plaira?"

Maestro: "Notice the menu items - croissants, pains au chocolat, and escargots en casque. These are common French dishes, but escargots (snails) might be new to some tourists."

Charles: "Je n'ai jamais essayé les escargots en casque. Est-ce que c'est bon?"

Sophie: "Oui, monsieur, ils sont délicieux. Mais si vous préférez quelque chose plus légère, nous avons également des salades fraîches."

Maestro: "Sophie is offering a lighter alternative to the snails - fresh salads. Notice the use of 'preferer' to express preference."

Charles: "Je vais essayer les escargots en casque. Quel est le prix, s'il vous plaît?"

Sophie: "C'est 12 euros, monsieur. Vous pouvez payer avec votre

In [14]:
unique_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
conversation_dir = os.path.join(config.GENERATED_OUTPUT_DIR, unique_id)
os.makedirs(conversation_dir, exist_ok=True)
final_audio_file = os.path.join(conversation_dir, "final_conversation.wav")


In [15]:
transcript_lines = reformat_transcript_to_list(explained_transcript)
transcript_lines

['Charles: "Bonjour, madame. Je suis un touriste et je suis un peu perdu dans votre menu. Pouvez-vous me donner quelques conseils?"',
 'Sophie: "Bien sûr, monsieur. Nous avons des croissants fraîchement cuits, des pains au chocolat et des escargots en casque. Quelque chose vous plaira?"',
 'Maestro: "Notice the menu items - croissants, pains au chocolat, and escargots en casque. These are common French dishes, but escargots (snails) might be new to some tourists."',
 'Charles: "Je n\'ai jamais essayé les escargots en casque. Est-ce que c\'est bon?"',
 'Sophie: "Oui, monsieur, ils sont délicieux. Mais si vous préférez quelque chose plus légère, nous avons également des salades fraîches."',
 'Maestro: "Sophie is offering a lighter alternative to the snails - fresh salads. Notice the use of \'preferer\' to express preference."',
 'Charles: "Je vais essayer les escargots en casque. Quel est le prix, s\'il vous plaît?"',
 'Sophie: "C\'est 12 euros, monsieur. Vous pouvez payer avec votre car

In [16]:
tts_processor = TTSProcessor(api_key=config.ELEVEN_API_KEY)

In [17]:
tts_processor.process_transcript(
    transcript_lines=transcript_lines,
    output_dir=conversation_dir,
    final_audio_file=final_audio_file,
    language=language,
    country=country_name,
    characters=characters 
    
)

print("Audio generated for each line, and combined audio saved at:")
print(final_audio_file)

Synthesizing line 1 for 'Charles' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 2 for 'Sophie' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 3 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 4 for 'Charles' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 5 for 'Sophie' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 6 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 7 for 'Charles' using voice 'hv6gVog5LgtIUX88Nmq8'...
Synthesizing line 8 for 'Sophie' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 9 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 10 for 'Charles' using voice 'hv6gVog5LgtIUX88Nmq8'...
Line 11 is not in expected format: (Charles paye et prend un siège)
Synthesizing line 12 for 'Sophie' using voice 'pVsdIxxCbRrbiwOhiRwg'...
Synthesizing line 13 for 'Maestro' using voice 'JBFqnCBsd6RMkjVDRZzb'...
Synthesizing line 14 for 'Charles' using voice 'hv6gVog5LgtIUX88Nmq8'